In [1]:
import requests
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [2]:
url = "https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Daerah_Khusus_Ibukota_Jakarta"

In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [4]:
tables = soup.find_all("table", class_="wikitable")

table_data = []

for table in tables:
    rows = table.find_all("tr")

    table_rows = []
    for row in rows:
        columns = row.find_all("td")

        row_data = []
        for column in columns:
            row_data.append(column.get_text(strip=True))

        table_rows.append(row_data)

    table_data.append(table_rows)

In [5]:
dfs = []
rows = []

for table in table_data:
    for row in table:
        if len(row) == 4:
            print(row)
            rows.append(row)
    print()


['31.71.05', 'Cempaka Putih', '3', 'Cempaka Putih BaratCempaka Putih TimurRawasari']
['31.71.01', 'Gambir', '6', 'CidengDuri PuloGambirKebon KelapaPetojo SelatanPetojo Utara']
['31.71.08', 'Johar Baru', '4', 'GalurJohar BaruKampung RawaTanah Tinggi']
['31.71.03', 'Kemayoran', '8', 'Cempaka BaruGunung Sahari SelatanHarapan MulyaKebon KosongKemayoranSerdangSumur BatuUtan Panjang']
['31.71.06', 'Menteng', '5', 'CikiniGondangdiaKebon SirihMentengPegangsaan']
['31.71.02', 'Sawah Besar', '5', 'Gunung Sahari UtaraKarang AnyarKartiniMangga Dua SelatanPasar Baru']
['31.71.04', 'Senen', '6', 'BungurKenariKramatKwitangPasebanSenen']
['31.71.07', 'Tanah Abang', '7', 'Bendungan HilirGeloraKampung BaliKaret TengsinKebon KacangKebon MelatiPetamburan']

['31.72.04', 'Cilincing', '7', 'CilincingKalibaruMarundaRorotanSemper BaratSemper TimurSukapura']
['31.72.06', 'Kelapa Gading', '3', 'Kelapa Gading BaratKelapa Gading TimurPegangsaan Dua']
['31.72.03', 'Koja', '6', 'KojaLagoaRawa Badak SelatanRawa Bad

In [6]:
geolocator = Nominatim(user_agent="my_user_agent")

def get_latitude_longitude(text):
    location = geolocator.geocode(text)
    address = location.raw
    latitude = address["lat"]
    longitude = address["lon"]
    latitude_longitude = f"{latitude}, {longitude}"
    return latitude_longitude

In [7]:
jkt_districts = pd.DataFrame(rows)[[0, 1]].iloc[:-2]
jkt_districts.columns = ["kemendagri_code", "district"]
jkt_districts["city"] = 8*["Jakarta Pusat"] + 6*["Jakarta Utara"] + 10*["Jakarta Timur"] + 10*["Jakarta Selatan"] + 8*["Jakarta Barat"]
jkt_districts["district_city"] = jkt_districts["district"] + ", " + jkt_districts["city"]
jkt_districts["latitude_longitude"] = jkt_districts["district_city"].apply(get_latitude_longitude)
jkt_districts

,kemendagri_code,district,city,district_city,latitude_longitude
0,31.71.05,Cempaka Putih,Jakarta Pusat,"Cempaka Putih, Jakarta Pusat","-6.181214499999999, 106.86854766095992"
1,31.71.01,Gambir,Jakarta Pusat,"Gambir, Jakarta Pusat","-6.1766841, 106.8306534"
2,31.71.08,Johar Baru,Jakarta Pusat,"Johar Baru, Jakarta Pusat","-6.1857706, 106.8567766"
3,31.71.03,Kemayoran,Jakarta Pusat,"Kemayoran, Jakarta Pusat","-6.1625464999999995, 106.85689034074326"
4,31.71.06,Menteng,Jakarta Pusat,"Menteng, Jakarta Pusat","-6.1950265, 106.83222419920054"
5,31.71.02,Sawah Besar,Jakarta Pusat,"Sawah Besar, Jakarta Pusat","-6.155891349999999, 106.83357979795008"
6,31.71.04,Senen,Jakarta Pusat,"Senen, Jakarta Pusat","-6.184971, 106.84323537475049"
7,31.71.07,Tanah Abang,Jakarta Pusat,"Tanah Abang, Jakarta Pusat","-6.2052581, 106.80949961953083"
8,31.72.04,Cilincing,Jakarta Utara,"Cilincing, Jakarta Utara","-6.12901455, 106.944453775"
9,31.72.06,Kelapa Gading,Jakarta Utara,"Kelapa Gading, Jakarta Utara","-6.15993785, 106.9024825763434"


In [8]:
jkt_districts.to_excel("jakarta_districts.xlsx", index=False)